<a href="https://colab.research.google.com/github/lukhnov/Azure-Databricks-Cookbook/blob/main/mdb_openai_assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



**First test of OpenAI Assitant flow**



In [ ]:
!pip install OpenAI
!pip install datasets

In [14]:
from google.colab import drive # mapping to a local drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
from google.colab import userdata
secret = userdata.get('openai')

In [42]:
from openai import OpenAI
import shelve
import time
client = OpenAI(api_key=secret)

In [ ]:
#uploading test dataset
from datasets import load_dataset
data = load_dataset("keivalya/MedQuad-MedicalQnADataset", split='train')
md_dataset = data.to_pandas()

In [45]:
#saving the dataset to GDrive
csv_file_path = '/content/drive/My Drive/Colab Notebooks/md_dataset.csv'
md_dataset.to_csv(csv_file_path, index=False)

In [46]:
# # uploading dataset to openai storage
# def upload_file(path):
#     # Upload a file with an "assistants" purpose
#     file = client.files.create(file=open(path, "rb"), purpose="assistants")
#     return file
# file2 = upload_file("/content/drive/My Drive/Colab Notebooks/md_dataset.csv")

In [51]:
# uploading dataset to openai storage
def upload_file(path):
    # Upload a file with an "assistants" purpose
    file = client.files.create(file=open(path, "rb"), purpose="assistants")
    return file
file = upload_file("/content/drive/My Drive/Colab Notebooks/med.pdf")

In [52]:
#creating assistant and assigning file to it
def create_assistant(file):
    assistant = client.beta.assistants.create(
        name="Compassionate Doctor",
        instructions="You are a very compassionate doctor. Using this context provided and any reasonable assumptions and inferences, attempt to address the user's questions in the most caring and respectful way",
        tools=[{"type": "retrieval"}],
        model="gpt-4-1106-preview",
        file_ids=[file.id],
    )
    return assistant
assistant = create_assistant(file)

In [53]:
#thread management
def check_if_thread_exists(wa_id):
    with shelve.open("threads_db") as threads_shelf:
        return threads_shelf.get(wa_id, None)


def store_thread(wa_id, thread_id):
    with shelve.open("threads_db", writeback=True) as threads_shelf:
        threads_shelf[wa_id] = thread_id

In [54]:
def generate_response(message_body, wa_id, name):
    # Check if there is already a thread_id for the wa_id
    thread_id = check_if_thread_exists(wa_id)

    # If a thread doesn't exist, create one and store it
    if thread_id is None:
        print(f"Creating new thread for {name} with wa_id {wa_id}")
        thread = client.beta.threads.create()
        store_thread(wa_id, thread.id)
        thread_id = thread.id

    # Otherwise, retrieve the existing thread
    else:
        print(f"Retrieving existing thread for {name} with wa_id {wa_id}")
        thread = client.beta.threads.retrieve(thread_id)

    # Add message to thread
    message = client.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content=message_body,
    )

    # Run the assistant and get the new message
    new_message = run_assistant(thread)
    print(f"To {name}:", new_message)
    return new_message

In [55]:
#run assistant
def run_assistant(thread):
    # Retrieve the Assistant
    assistant = client.beta.assistants.retrieve("asst_zOGkjBPikc4xiSrALsHeRzWn")

    # Run the assistant
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id,
    )

    # Wait for completion
    while run.status != "completed":
        # Be nice to the API
        time.sleep(0.5)
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

    # Retrieve the Messages
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    new_message = messages.data[0].content[0].text.value
    print(f"Generated message: {new_message}")
    return new_message

In [58]:
new_message = generate_response("What is scoping review methodology?", "123", "John")

Retrieving existing thread for John with wa_id 123
Generated message: A scoping review methodology is a type of research strategy used to map the key concepts underpinning a research area and the main sources and types of evidence available. It is often undertaken before a full systematic review and can be used for several purposes, including:

1. **Examining the extent, range, and nature of research activity**.
2. **Determining the value of undertaking a full systematic review**.
3. **Summarizing and disseminating research findings**.
4. **Identifying gaps in the existing literature**.

Scoping reviews differ from systematic reviews primarily in their objectives and methodology. While systematic reviews typically focus on a specific question and involve a detailed search, critical appraisal, and synthesis of literature to answer that question, scoping reviews are broader in nature, aiming to provide an overview of the available research without the detailed synthesis or quality assess

In [59]:
new_message = generate_response("What is caused by asospasm of peripheral arteries", "123", "John")


Retrieving existing thread for John with wa_id 123
Generated message: Vasospasm of peripheral arteries, specifically, is referred to in the context of Raynaud's disease, which is caused by intense vasospasm of peripheral arteries. Raynaud's disease is a rare disorder that affects the arteries, which are blood vessels that carry blood from your heart to different parts of your body .

The causes of Raynaud's disease can include:

- Diseases and conditions that directly damage the arteries or damage the nerves that control the arteries in the hands and feet.
- Repetitive actions that damage the nerves that control the arteries in the hands and feet.
- Injuries to the hands and feet.
- Exposure to certain chemicals, including beta-adrenoceptor ergotamine and derivatives.
- Medications that narrow the arteries or affect blood pressure.
- Occupational exposure to vibrating tools and cold.
- Cryoglobulinemia.
- Conditions that are part of the CREST syndrome.

The clinical features associated